In [181]:
import pandas as pd
import numpy as np
import networkx as nx
import requests
from bs4 import BeautifulSoup
import cPickle as pickle
from model.utilities import haversine, diff_timestamps

In [180]:
reload(utilities)

<module 'model.utilities' from 'model/utilities.py'>

In [33]:
stops = pd.read_csv('data/google_transit/stops.txt', index_col='stop_id')
routes = pd.read_csv('data/google_transit/routes.txt', index_col='route_id')
trips = pd.read_csv('data/google_transit/trips.txt', index_col='trip_id')
stop_times = pd.read_csv('data/google_transit/stop_times.txt')
shapes = pd.read_csv('data/google_transit/shapes.txt')

In [216]:
trips.head()

,route_id,service_id,trip_headsign,direction_id,block_id,shape_id
trip_id,,,,,,
6858659,11047,2,Geary + 33rd Avenue,0,115,135594
7048008,11047,1,Geary + 33rd Avenue,0,110,135595
7048006,11047,1,Geary + 33rd Avenue,0,116,135595
7048005,11047,1,Geary + 33rd Avenue,0,103,135595
7048004,11047,1,Geary + 33rd Avenue,0,113,135595


In [189]:
stops[stops['stop_name'] == 'Not a public stop']

,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url
stop_id,,,,,,
7520,Not a public stop,,37.709523,-122.404631,,
7530,Not a public stop,,37.755387,-122.386889,,
7531,Not a public stop,,37.755387,-122.386832,,
7540,Not a public stop,,37.755387,-122.386867,,


In [203]:
stop_times[stop_times['stop_id'] == 7520]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
126835,6869040,06:55:00,06:55:00,7520,43,,,,
126876,6869041,07:08:00,07:08:00,7520,43,,,,
126917,6869042,07:22:00,07:22:00,7520,43,,,,
126958,6869043,07:35:00,07:35:00,7520,43,,,,
126999,6869044,07:49:00,07:49:00,7520,43,,,,
127040,6869045,08:03:00,08:03:00,7520,43,,,,
127081,6869046,08:17:00,08:17:00,7520,43,,,,
127122,6869047,08:30:00,08:30:00,7520,43,,,,
127163,6869048,08:42:00,08:42:00,7520,43,,,,
127204,6869049,08:54:00,08:54:00,7520,43,,,,


In [206]:
stop_times[stop_times['trip_id'] == 6869064]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
127779,6869064,11:03:00,11:03:00,7227,2,,,,
127780,6869064,11:06:05,11:06:05,5657,4,,,,
127781,6869064,11:07:43,11:07:43,5639,5,,,,
127782,6869064,11:09:13,11:09:13,5678,6,,,,
127783,6869064,11:11:00,11:11:00,5694,7,,,,
127784,6869064,11:12:21,11:12:21,5655,8,,,,
127785,6869064,11:13:42,11:13:42,5695,9,,,,
127786,6869064,11:15:02,11:15:02,5656,10,,,,
127787,6869064,11:16:11,11:16:11,5676,11,,,,
127788,6869064,11:17:19,11:17:19,5680,12,,,,


In [212]:
stops = stops[~stops.index.isin([7520, 7530, 7531, 7540])]

In [214]:
stop_times = stop_times[~stop_times['stop_id'].isin([7520, 7530, 7531, 7540])]

In [215]:
stop_times[stop_times['stop_id'].isin([7520, 7530, 7531, 7540])]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled


In [69]:
G_x = nx.DiGraph()

In [220]:
trips.loc[6869064]['shape_id']

135651

In [70]:
# RIDING GRAPH
# For each trip, for each stop, add the stop as a node
# and connect it to the previous stop in the trip
for trip_id in trips.index:
    trip_stops = stop_times[stop_times['trip_id'] == trip_id]
    trip_info = trips.loc[trip_id]
    route_id = trip_info['route_id']
    service_id = trip_info['service_id']
    block_id = trip_info['block_id']   
    shape_id = trip_info['shape_id']
    for i, row in enumerate(trip_stops.iterrows()):
        stop_id = row[1]['stop_id']
        arrival_time = row[1]['arrival_time']
        stop_info = stops.loc[stop_id]
        stop_name = stop_info['stop_name']
        stop_lat = stop_info['stop_lat']
        stop_lon = stop_info['stop_lon']        
        stop_node = '{0}_{1}'.format(stop_id, arrival_time)
        if not G_x.has_node(stop_node):
            G_x.add_node(stop_node, stop_id=stop_id, stop_name=stop_name,\
                         stop_lat=stop_lat, stop_lon=stop_lon, arrival_time=arrival_time)
        if i == 0:
            last_stop = stop_node
            continue
        ride_time = diff_timestamps(G_x.node[last_stop]['arrival_time'],arrival_time)
        G_x.add_edge(last_stop, stop_node, trip_id=trip_id, route_id=route_id, service_id=service_id, \
                     block_id=block_id, shape_id=shape_id, type=ride, duration=ride_time)
        last_stop = stop_node

In [182]:
G_x.node['3003_05:51:12']['arrival_time']

'05:51:12'

In [184]:
diff_timestamps(G_x.node['3003_05:51:12']['arrival_time'],'08:53:52')

10960

In [117]:
int(G_x.node['3003_05:51:12']['arrival_time'][:2]) > 6

False

In [109]:
from datetime import datetime as dt
#s1 = '23:51:12'
#s2 = '01:15:49' # for example
#FMT = '%H:%M:%S'
#tdelt = datetime.strptime(s2, FMT) - datetime.strptime(s1, FMT)

In [106]:
tdelt.seconds

5077

In [131]:
node_time = '24:15:02'

In [132]:
node_time = '0' + str(int(node_time[:2])%24) + node_time[2:]

In [133]:
node_time

'00:15:02'

In [81]:
# WAITING GRAPH
# For each stop_id, connect each timepoint to the next
sorted_nodes = sorted(G_x.nodes())

In [86]:
len(sorted_nodes)

749201

In [91]:
sorted_nodes[:22]

['3003_05:25:12',
 '3003_05:51:12',
 '3003_06:11:46',
 '3003_06:32:46',
 '3003_06:54:46',
 '3003_06:56:12',
 '3003_07:11:18',
 '3003_07:16:12',
 '3003_07:26:18',
 '3003_07:36:12',
 '3003_07:43:18',
 '3003_07:56:12',
 '3003_07:59:18',
 '3003_08:06:18',
 '3003_08:14:18',
 '3003_08:16:46',
 '3003_08:21:18',
 '3003_08:29:18',
 '3003_08:36:18',
 '3003_08:36:46',
 '3003_08:44:18',
 '3003_08:53:52']

In [151]:
for i, node in enumerate(sorted_nodes):
    if i == 0:
        last_node = node
        continue
    node_id = G_x.node[node]['stop_id']
    node_time = G_x.node[node]['arr_time']
    last_node_id = G_x.node[last_node]['stop_id']
    last_node_time = G_x.node[last_node]['arrival_time']
    wait_time = diff_timestamps(last_node_time, node_time)
    if node_id == last_node_id:
        G_x.add_edge(last_node, node, duration=wait_time, type=wait)
    last_node = node

In [188]:
G_x['3003_05:25:12']

{'3003_05:51:12': {'duration': 1560, type: 'wait'},
 '3009_05:26:27': {'block_id': 1001,
  'route_id': 11057,
  'service_id': 1,
  'shape_id': 138190,
  'trip_id': 7041614,
  type: 'ride'}}

In [156]:
for u, v, d in G_x.edges_iter(data=True):
    if d.has_key('duration'):
        d[type] = 'wait'
    else:
        d[type] = 'ride'

In [157]:
G_x.edge['3003_05:25:12']

{'3003_05:51:12': {'duration': 1560, type: 'wait'},
 '3009_05:26:27': {'block_id': 1001,
  'route_id': 11057,
  'service_id': 1,
  'shape_id': 138190,
  'trip_id': 7041614,
  type: 'ride'}}

In [153]:
nx.write_gpickle(G_x, 'graph_x_3.gpkl')

In [75]:
nx.write_gpickle?

In [38]:
stops.loc[98]['stop_name']

'2ND ST & MARKET ST'

In [9]:
foo = stop_times[stop_times['trip_id'] == 7048004]

In [18]:
for buh in foo.iterrows():
    print stops[stops['stop_id'] == buh[1]['stop_id']]['stop_name'].iloc[0]

Clay St & Drumm St
Sacramento St & Davis St
Sacramento St & Battery St
Sacramento St & Sansome St
Sacramento St & Montgomery St
Sacramento St & Kearny St
Sacramento St & Grant Ave
Sacramento St & Stockton St
Sacramento St & Powell St
Sacramento St & Sproule Ln
Sacramento St & Jones St
Sacramento St & Leavenworth St
Sacramento St & Hyde St
Sacramento St & Larkin St
Sacramento St & Polk St
Sacramento St & Van Ness Ave
Sacramento St & Franklin St
Sacramento St & Gough St
Sacramento St & Octavia St
Sacramento St & Laguna St
Sacramento St & Buchanan St
Sacramento St & Webster St
Sacramento St & Fillmore St
Steiner St & California St
California St & Pierce St
California St & Divisadero St
California St & Baker St
California St & Presidio Ave
California St & Laurel St
California St & Spruce St
California St & Maple St
California St & Cherry St
California St & Arguello Blvd
California St & 4th Ave
California St & 6th Ave
California St & 8th St
California St & 10th Ave
California St & 12th Ave


In [58]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,6858631,26:00:00,26:00:00,4015,1,,,,
1,6858631,26:00:45,26:00:45,6294,2,,,,
2,6858631,26:01:33,26:01:33,6290,3,,,,
3,6858631,26:02:00,26:02:00,6314,4,,,,
4,6858631,26:02:35,26:02:35,6307,5,,,,


In [ ]:
G_x = nx.DiGraph()